# Benchmark with conditional formating
All we need in the protocal is the Haar-random unitaries. A random unitary can be represeted by rotation matrices like this:

In this project we benchmark with those conditional probabilities

In [1]:
from pyquil import get_qc, Program
from pyquil.gates import *
from pyquil.api import local_forest_runtime
from pyquil.quilbase import Declare
from pyquil.simulation.tools import lifted_gate, program_unitary
from pyquil.quil import *

In [2]:
import numpy as np
import math
from math import pi
import random
import copy
from tqdm import tqdm_notebook as tqdm

In [3]:
from functions import *

In [4]:
if __name__ == "__main__":
    target_qubits = [0,1]

#     First step choose m and the K_m sequences of 
    m = 1
    k_m = 5 #n. of diff sequences
    n_m = 5  #n. of samples from a certain sequence


In [5]:
def universal_two_qubits_packs_generator(qmachine, target_qubits:list, num_layer:int):
    list_gates = []
    for index in range(num_layer):
        draft_circuit = give_random_two_qubit_circuit(target_qubits)
        list_gates.extend( qmachine.compiler.quil_to_native_quil(draft_circuit) )
    list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
    return list_gates

In [10]:
def machine_response_rb_universal_two_qubits_conditional(qmachine, target_qubits:list, m:int, k_m, n_m):
    """
    It samples and record the accept or reject of the machine with native gates chosen with conditions for rigetti.
    ::return response_matrix including accepts and rejects in columns
    """
    num_qubits = len(target_qubits)
    response_matrix = np.zeros((k_m,n_m))
    
    for i_sequ in tqdm(range(k_m), desc = 'Sequences'):
        gate_list = universal_two_qubits_packs_generator(qmachine, target_qubits, m)
        prog = Program() #All qubits begin with |0> state
        
        for gate in gate_list:
            prog += gate
        
        #Come back to our initial state
#         for gate in reversed(gate_list):
# #             prog += copy.deepcopy(gate).dagger() #dagger has replacing operations
#             gate_daggered = copy.deepcopy(gate)
#             gate_daggered.params[0] *= -1 #make daggered rotation 
#             prog += gate_daggered
        u_inverse_definition = DefGate('U_inverse', np.linalg.inv(program_unitary(prog, n_qubits=2)))
        U_inverse = u_inverse_definition.get_constructor()
        
        prog += u_inverse_definition
        prog += qmachine.compiler.quil_to_native_quil(Program(U_inverse(*target_qubits)))
        
        #Do not let the quilc to alter the gates by optimization
#         prog = Program('PRAGMA PRESERVE_BLOCK') + prog
#         prog += Program('PRAGMA END_PRESERVE_BLOCK')
        
        #Measurments
        ro = prog.declare('ro', 'BIT', num_qubits)
        for q in range(num_qubits):
            prog += MEASURE(q, ro[q])
        prog = prog.wrap_in_numshots_loop(n_m)

        #Run the program
        executable = qmachine.compile(prog)
        result = qmachine.run(executable)
        measured_outcome = result.readout_data.get('ro')

        response_matrix[i_sequ,:] = 1 - np.bool_(np.sum(measured_outcome, axis = 1)) # 1 if it is equal to n_zero state
    return response_matrix

In [11]:
if __name__ == "__main__":
#     qc = get_qc( str(num_qubits) + 'q-qvm')  # You can make any 'nq-qvm'
    qc = get_qc("9q-square-noisy-qvm")
    response = machine_response_rb_universal_two_qubits_conditional(qc, [0,1], m, k_m, n_m)


C:\Users\mohsen\AppData\Local\Temp\ipykernel_21344\3813690421.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i_sequ in tqdm(range(k_m), desc = 'Sequences'):


Sequences:   0%|          | 0/5 [00:00<?, ?it/s]

RPCError: Unhandled error in host program:
Requested to rewire U_inverse 0 1, but we don't know how to do this.

In [19]:
# if __name__ == "__main__":
#     !jupyter nbconvert RB_with_Rigetti_native_gates_conditional_probability_two_qubits.ipynb --to python

In [20]:
print(len(executable))

41


In [21]:
print(executable)

DEFGATE U_inverse:
    -0.060758970263272115+0.3369153865169795i, 0.7369819335893378-0.18176139613229572i, 0.5245981539189907+0.09264896319550353i, 0.09667506378313864+0.11611921833068549i
    0.30716818143452196-0.18350210473833664i, -0.06250040074351415-0.013034634525186843i, 0.4029871329528485+0.14628740748650398i, -0.8026446383043141-0.1996532838877445i
    0.525379165048792+0.49199519458803836i, 0.08086120153326942+0.5265135168784146i, -0.11167345918824813-0.4203408903733145i, -0.07378819845597273+0.059668419013329796i
    0.009627877927138229-0.4864111646878594i, 0.02539368379142779+0.3678917858597443i, 0.4782867397067181-0.3383765367497009i, 0.37869117761053017-0.37504450876128465i

DECLARE ro BIT[2]
RZ(-0.5746488128618452) 0
RX(pi/2) 0
RZ(2.3081088883517342) 0
RX(-pi/2) 0
RZ(-0.4736209864636214) 0
RZ(-0.3817309888514139) 1
RX(pi/2) 1
RZ(1.0269783330927444) 1
RX(-pi/2) 1
RZ(-2.9248592209925763) 1
XY(pi) 1 0
RZ(pi/2) 0
RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
RZ(-2.7889649067582885) 0
R